## Build an End-to-End System
This puts together the chain of prompts that you saw throughout the course.

In [1]:
#!/usr/bin/env python3
import os
from dotenv import load_dotenv, find_dotenv
import streamlit as st
from openai import OpenAI

# Load environment variables
load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

# Create OpenAI client
client = OpenAI(api_key=api_key)

# Chat completion
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! Can you recommend a product?"}
    ]
)

print(response.choices[0].message.content)



Of course! I'd be happy to help you find a product. What type of product are you looking for, and do you have any specific preferences or requirements in mind?


### call to Open AI

In [2]:
# Function to get a completion from messages
def get_completion_from_messages(messages, model="gpt-4", temperature=0.7, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content
# Function to read a string representation of a list of dictionaries

### System of chained prompts for processing the user query

In [3]:
# Function to get all products and categories
def get_products_and_category():
    # Replace with your real product list if needed
    return ["smartx pro phone", "fotosnap camera", "dslr", "tv", "tvs"]

# Function to extract matched products from user input
def find_category_and_product_only(user_input, product_list):
    user_input = user_input.lower()
    matched = [product for product in product_list if product.lower() in user_input]
    return matched if matched else ["No match found"]

# Function to convert a list of matched products into a list format
def read_string_to_list(category_and_product_response):
    # Already a list, so just return it
    return category_and_product_response

# Function to generate a final output string from a list of categories and products
def generate_output_string(category_and_product_list):
    return "\n".join(str(x) for x in category_and_product_list)

# Example usage
if __name__ == "__main__":
    user_input = "I'm interested in a DSLR and a TV"
    product_list = get_products_and_category()
    matched_products = find_category_and_product_only(user_input, product_list)
    product_list_output = read_string_to_list(matched_products)
    final_output = generate_output_string(product_list_output)
    print(final_output)


dslr
tv


### Function that collects user and assistant messages over time

In [5]:
# Example of integrating with a simple UI (e.g., Jupyter notebook with ipywidgets)
import ipywidgets as widgets
from IPython.display import display

inp = widgets.Text(
    value='',
    placeholder='Type your message here',
    description='You:',
    disabled=False
)

send_button = widgets.Button(description="Send")

context = []

def collect_messages(debug=False):
    global inp, context
    user_input = inp.value
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''  # ✅ Clears the field

    response, context = process_user_message(user_input, context, debug=True)
    context.append({'role': 'assistant', 'content': f"{response}"})

send_button.on_click(lambda b: collect_messages(debug=True))

display(inp, send_button)

Text(value='', description='You:', placeholder='Type your message here')

Button(description='Send', style=ButtonStyle())

### Chat with the chatbot!¶
Note that the system message includes detailed instructions about what the OrderBot should do.

In [6]:
# filename: chatbot.py


st.set_page_config(page_title="Chat with the Chatbot!", page_icon="🤖")

st.title("Chat with the Chatbot! 🤖")
st.markdown("*(System message includes detailed instructions about OrderBot)*")

# Initialize chat history in session state
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "You are Service Assistant"}
    ]

# User input
user_input = st.text_input("You:", key="user_input", placeholder="Enter text here...")

# On send button click
if st.button("Send", key="send_button") and user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Real bot response using your assistant logic
    response, updated_messages = process_user_message(user_input, st.session_state.messages[1:], debug=False)
    st.session_state.messages = [{"role": "system", "content": "You are Service Assistant"}] + updated_messages

    st.rerun()


    # Clear input
    st.experimental_rerun()

# Display chat history
for msg in st.session_state.messages[1:]:  # skip system message
    if msg["role"] == "user":
        st.markdown(f"**You:** {msg['content']}")
    else:
        st.markdown(f"**Service Assistant:** {msg['content']}")



2025-09-24 11:15:57.989 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-24 11:15:57.991 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-24 11:15:58.241 
  command:

    streamlit run /home/oliver/.venv/lib64/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-24 11:15:58.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-24 11:15:58.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-24 11:15:58.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-24 11:15:58.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored wh

In [8]:
# FINAL WORKING VERSION - Add this as a NEW CELL

# Product database
def get_products_and_category_v2():
    return {
        "smartx pro phone": "SmartX Pro Phone - $899 - Premium smartphone with 6.1-inch display, 128GB storage, triple camera system, 5G enabled",
        "fotosnap camera": "FotoSnap DSLR Camera - $1,299 - Professional DSLR with 24.2MP sensor, 4K video, weather sealed", 
        "fotosnap compact": "FotoSnap Compact Camera - $599 - Portable camera with 20MP sensor, 10x optical zoom, WiFi enabled",
        "dslr": "FotoSnap DSLR Camera - $1,299 - Professional DSLR with 24.2MP sensor, 4K video, weather sealed",
        "tv": "TCL 55-inch Smart TV - $649 - 4K Ultra HD, HDR support, Smart TV platform",
        "tvs": "We offer TCL 55-inch Smart TV ($649) and Samsung 65-inch QLED TV ($1,199)"
    }

# Product matching  
def find_category_and_product_v2(user_input, products_dict):
    user_input = user_input.lower()
    matched = []
    for product_key, product_info in products_dict.items():
        if product_key.lower() in user_input:
            matched.append(product_info)
    return matched if matched else ["No specific products found"]

# FORCE use of the REAL OpenAI function from cell 2
def get_real_completion(messages, model="gpt-3.5-turbo", temperature=0.7, max_tokens=500):
    """This ensures we use the real OpenAI API"""
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Main processing function - WORKING VERSION
def process_user_message_v2(user_input, all_messages, debug=True):
    if debug: print("Step 1: Input passed moderation check.")
    
    # Get products
    products_dict = get_products_and_category_v2()
    matched_products = find_category_and_product_v2(user_input, products_dict)
    if debug: print("Step 2: Extracted list of products.")

    # Generate product information
    product_information = "\n".join(matched_products)
    if debug: print("Step 3: Looked up product information.")

    # Create system message
    system_message = f"""
You are a sales representative working at TechStore Electronics. 
You work in our physical store and you DO have inventory and products to sell.

Our current inventory includes:
{product_information}

IMPORTANT RULES:
- Never say "I don't have inventory" or "I'm an AI" or "As an artificial intelligence"
- Always act as a human store employee selling these specific products
- Focus on our products with prices and features
- Be helpful and sales-focused
- Ask customers follow-up questions

When customers greet you or ask about products, respond as a store employee would.
"""
    
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_input}
    ]

    # Use the real OpenAI function
    final_response = get_real_completion(messages)
    if debug: print("Step 4: Generated response to user question.")
    
    if debug: print("Step 5-7: All checks passed.")
    
    # Update conversation history
    updated_messages = all_messages + [
        {'role': 'user', 'content': user_input},
        {'role': 'assistant', 'content': final_response}
    ]
    
    return final_response, updated_messages

# Test the working version
print("Testing the working version:")
response, _ = process_user_message_v2("Hi, do you have information about DSLR?", [])
print(f"Response: {response}")

Testing the working version:
Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5-7: All checks passed.
Response: Hello! Yes, we have the FotoSnap DSLR Camera available. It is a professional DSLR with a 24.2MP sensor, 4K video capability, and it's weather sealed. The price for the FotoSnap DSLR Camera is $1,299. Is there anything specific you would like to know about this camera?
